# PySpark Cluster Analysis on Weather Data

## HPC and Data Science Summer Institute

Mai H. Nguyen, UC San Diego

---
Resources:
- Spark DataFrame Guide: https://spark.apache.org/docs/latest/sql-programming-guide.html
- PySpark API Documentation: https://spark.apache.org/docs/latest/api/python/index.html
- PySpark Cheat Sheet PDF

## Setup 

In [1]:
# Start Spark session
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAll([('spark.master', 'local[2]'),
                                   ('spark.app.name', 'PySpark Cluster Analysis')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

print (spark.version)
print (pyspark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/02 07:25:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.1
<module 'pyspark.version' from '/opt/spark/3.5.1/python/lib/pyspark.zip/pyspark/version.py'>


In [2]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd

# Show plots in notebook
%matplotlib inline

## Read in data

Data source:  http://hpwren.ucsd.edu

In [3]:
from pyspark.sql.types import StructType, StructField 
from pyspark.sql.types import StringType, IntegerType, DoubleType, TimestampType

# Specify schema
schema = StructType ([ \
    StructField ("rowID",IntegerType(),True), \
    StructField ("hpwren_timestamp",TimestampType(),True), \
    StructField ("air_pressure",DoubleType(),True), \
    StructField ("air_temp",DoubleType(),True), \
    StructField ("avg_wind_direction",DoubleType(),True), \
    StructField ("avg_wind_speed",DoubleType(),True), \
    StructField ("max_wind_direction",DoubleType(),True), \
    StructField ("max_wind_speed",DoubleType(),True), \
    StructField ("min_wind_direction",DoubleType(),True), \
    StructField ("min_wind_speed",DoubleType(),True), \
    StructField ("rain_accumulation",DoubleType(),True), \
    StructField ("rain_duration",DoubleType(),True), \
    StructField ("relative_humidity",DoubleType(),True)
])

In [4]:
# Read in data and put in Spark DataFrame

from os.path import expanduser
HOME = expanduser("~")

data_path = HOME + '/data/'
inputfile = data_path + "minute_weather.csv"
print(inputfile)

df = spark.read.csv (inputfile, header=True, schema=schema).cache()

/home/gseo/data/minute_weather.csv


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/gseo/data/minute_weather.csv.

## Explore data

### Print schema

In [ ]:
df.printSchema()

### Count rows

In [ ]:
df.count()

### Show summary statistics

In [ ]:
df.describe().toPandas().transpose()

## Prepare data

### Drop nulls

In [ ]:
workingDF = df.na.drop()
workingDF.count()

### Create feature vector

In [ ]:
from pyspark.ml.feature import VectorAssembler

featuresUsed = ['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction', 
        'max_wind_speed','relative_humidity']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(workingDF)

In [ ]:
assembled.show(1)

### Scale data

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

In [ ]:
scaledData.show(1)

## Perform cluster analysis

### Generate elbow plot to determine value(s) for k

In [ ]:
%%time
import utils

# Only need to run this once to find value(s) to try for k
# Set to False if already know value for k
create_elbow_plot = True 

# Get elbow plot using subset of data
if create_elbow_plot == True:
    sampledData = scaledData.filter((scaledData.rowID % 20) == 0).select("features").cache() 
    k_attempts = range(5,15)
    print('Trying k from {} to {} with {} samples\n'.format(list(k_attempts)[0],
                                                          list(k_attempts)[-1], 
                                                          sampledData.count()))
    wsseList = utils.elbow(sampledData, k_attempts)
    utils.elbow_plot(wsseList, k_attempts)

### Perform Clustering Using K-Means 

In [ ]:
scaledData.printSchema()

In [ ]:
from pyspark.ml.clustering import KMeans

scaledDataFeat = scaledData.select("features").cache()

# Set number of clusters
nClusters = 11

kmeans = KMeans(k=nClusters, seed=1)
model = kmeans.fit(scaledDataFeat)

In [ ]:
# Show cluster centers

centers = model.clusterCenters()
pd.DataFrame(centers,columns=featuresUsed)

In [ ]:
# Show cluster sizes

model.summary.clusterSizes

## Generate cluster profile plots

In [ ]:
centersNamed = utils.pd_centers(featuresUsed,centers)
print(centersNamed.columns.values)

### Profiles for All Clusters

This is a parallel plot to show the cluster centers for all clusters.  
Data samples were standardized to have 0-mean and 1-standard-deviation before k-means was applied, so values of cluster centers can be interpreted as standard deviations from the mean.  

In [ ]:
numClusters = len(centersNamed.index)
colors_used = utils.parallel_plot(centersNamed, numClusters)

### Clusters Capturing Dry Days

Clusters with lower-than-average relative_humidity values capture dry days

In [ ]:
utils.parallel_plot(centersNamed[centersNamed['relative_humidity'] < -0.5], 
                   numClusters, colors=colors_used);

### Clusters Capturing Humid Days

Clusters with higher-than-average relative_humidity values capture humid days

In [ ]:
utils.parallel_plot(centersNamed[centersNamed['relative_humidity'] > 0.5], 
                    numClusters, colors=colors_used);

### Clusters Capturing Hot Days

In [ ]:
utils.parallel_plot(centersNamed[centersNamed['air_temp'] > 0.5], 
                   numClusters, colors=colors_used);

### Clusters Capturing Windy Days

In [ ]:
utils.parallel_plot(centersNamed[centersNamed['max_wind_speed'] > 0.5], 
                   numClusters, colors=colors_used);

## Save Model

In [ ]:
model.write().overwrite().save("weather-kmeans-model")

## Stop Spark session

In [ ]:
spark.stop()